In [8]:
import cifar10,cifar10_input
import tensorflow as tf
import numpy as np
import time

In [9]:
max_steps = 3000
batch_size = 128
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'

In [10]:
def variable_with_weight_loss(shape,stddev,wl):
    var = tf.Variable(tf.truncated_normal(shape,stddev = stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var),wl,name='weight_loss')
        tf.add_to_collection('losses',weight_loss)
    return var

In [11]:
cifar10.maybe_download_and_extract()

PermissionError: [Errno 13] Permission denied: '/tmp/cifar10_data/cifar-10-batches-bin'

In [6]:
images_train,labels_train = cifar10_input.distorted_inputs(data_dir=data_dir,batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [7]:
images_test,labels_test = cifar10_input.inputs(eval_data=True, data_dir=data_dir,batch_size=batch_size)

In [8]:
image_holder = tf.placeholder(tf.float32,[batch_size,24,24,3])

label_holder = tf.placeholder(tf.int32, [batch_size])

In [9]:
weight1 = variable_with_weight_loss(shape=[5,5,3,64],stddev=53-2,wl=0.0)
kernel1 = tf.nn.conv2d(image_holder,weight1,[1,1,1,1],padding='SAME')
bias1 = tf.Variable(tf.constant(0.0,shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1,bias1))
pool1 = tf.nn.max_pool(conv1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
norm1 = tf.nn.lrn(pool1,4,bias=1.0,alpha=0.001/9.0,beta=0.75)

In [10]:
weight2 = variable_with_weight_loss(shape=[5,5,64,64],stddev=5e-2,wl=0.0)
kernel2 = tf.nn.conv2d(norm1,weight2,[1,1,1,1],padding='SAME')
bias2 = tf.Variable(tf.constant(0.1,shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2,bias2))
norm2 = tf.nn.lrn(conv2,4,bias=1.0,alpha=0.001/9.0,beta=0.75)
pool2 = tf.nn.max_pool(norm2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')

In [11]:
reshape = tf.reshape(pool2,[batch_size,-1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim,384],stddev=0.04,wl=0.004)
bias3 = tf.Variable(tf.constant(0.1,shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape,weight3)+bias3)

In [12]:
weight4 = variable_with_weight_loss(shape=[384,192],stddev=0.04,wl=0.04)
bias4 = tf.Variable(tf.constant(0.1,shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3,weight4)+bias4)

In [13]:
weight5 = variable_with_weight_loss(shape=[192,10],stddev=0.04,wl=0.04)
bias5 = tf.Variable(tf.constant(0.0,shape=[10]))
logits = tf.add(tf.matmul(local4,weight5),bias5)

In [14]:
def loss(logits,labels):
    labels = tf.cast(labels,tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels,name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'),name='total_loss')

In [15]:
loss = loss(logits,label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [16]:
top_k_op = tf.nn.in_top_k(logits,label_holder,1)

In [17]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [18]:
tf.train.start_queue_runners()

[<Thread(Thread-4, started daemon 140681100121856)>,
 <Thread(Thread-5, started daemon 140681091729152)>,
 <Thread(Thread-6, started daemon 140681083336448)>,
 <Thread(Thread-7, started daemon 140681074943744)>,
 <Thread(Thread-8, started daemon 140681066551040)>,
 <Thread(Thread-9, started daemon 140680110274304)>,
 <Thread(Thread-10, started daemon 140680101881600)>,
 <Thread(Thread-11, started daemon 140680093488896)>,
 <Thread(Thread-12, started daemon 140680085096192)>,
 <Thread(Thread-13, started daemon 140680076703488)>,
 <Thread(Thread-14, started daemon 140680068310784)>,
 <Thread(Thread-15, started daemon 140680059918080)>,
 <Thread(Thread-16, started daemon 140679573403392)>,
 <Thread(Thread-17, started daemon 140679565010688)>,
 <Thread(Thread-18, started daemon 140679556617984)>,
 <Thread(Thread-19, started daemon 140679548225280)>,
 <Thread(Thread-20, started daemon 140679539832576)>,
 <Thread(Thread-21, started daemon 140679531439872)>,
 <Thread(Thread-22, started daemon

In [19]:
for step in range(max_steps):
    start_time = time.time()
    image_batch,label_batch = sess.run([images_train,labels_train])
    _,loss_value = sess.run([train_op,loss],feed_dict={image_holder:image_batch,label_holder:label_batch})
    duration = time.time() - start_time
    if step%10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)
        format_str=('step %d,loss%.2f(%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step,loss_value,examples_per_sec,sec_per_batch))

step 0,loss9.63(15.5 examples/sec; 8.277 sec/batch)
step 10,loss5.99(2200.9 examples/sec; 0.058 sec/batch)
step 20,loss5.62(2489.7 examples/sec; 0.051 sec/batch)
step 30,loss5.18(2489.3 examples/sec; 0.051 sec/batch)
step 40,loss4.89(2564.3 examples/sec; 0.050 sec/batch)
step 50,loss4.84(2284.5 examples/sec; 0.056 sec/batch)
step 60,loss4.64(1757.0 examples/sec; 0.073 sec/batch)
step 70,loss4.50(2351.0 examples/sec; 0.054 sec/batch)
step 80,loss4.34(2431.9 examples/sec; 0.053 sec/batch)
step 90,loss4.16(2482.4 examples/sec; 0.052 sec/batch)
step 100,loss4.11(2583.6 examples/sec; 0.050 sec/batch)
step 110,loss4.02(1316.3 examples/sec; 0.097 sec/batch)
step 120,loss3.88(2493.5 examples/sec; 0.051 sec/batch)
step 130,loss3.79(1689.0 examples/sec; 0.076 sec/batch)
step 140,loss3.61(2557.8 examples/sec; 0.050 sec/batch)
step 150,loss3.70(2344.3 examples/sec; 0.055 sec/batch)
step 160,loss3.69(847.0 examples/sec; 0.151 sec/batch)
step 170,loss3.67(2499.3 examples/sec; 0.051 sec/batch)
step 1

In [20]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))
true_count = 0
total_sample_out = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch,label_batch = sess.run([images_test,labels_test])
    predictions = sess.run([top_k_op],feed_dict={image_holder:image_batch,label_holder:label_batch})
    true_count += np.sum(predictions)
    step += 1

In [21]:
precision = true_count / total_sample_out
print('precision @ 1 = %.3f'%precision)

precision @ 1 = 0.531


In [22]:
1018*0.002

2.036